In [ ]:
import re
import ast

data_filtered = pd.read_csv(r"C:\Users\Gav\Anaconda3\Projects\Jobs_scraping\linkedin-jobs-2021-07-predicted.csv")

data_filtered['yrs'] = data_filtered['description'].str.findall(r"([\d. +-/]+|[\(\d\). +-/]+|[\w\+\b]+)\s*years of")
raw = data_filtered['yrs']

data_filtered = data_filtered[~data_filtered.index.duplicated(keep='first')]

data_filtered['yrs'] = data_filtered['yrs'].astype(str)\
         .str.replace(".", ' ')\
.str.replace("(", '')\
.str.replace(")", '')\
.str.replace("+", ' ')\
.str.replace("œ", ' ')\
.str.replace("(\d-)", ' ',regex=True)\
.str.replace("(\s-\s\d+)", '',regex=True)\
.str.replace("(-)", '',regex=True)\
.str.replace("(,1\d|,10|ten|Ten)", '10',regex=True)\
.str.replace("(,5|five|œ5|Five)", '5',regex=True)\
.str.replace("eighteen", '18',regex=True)\
.str.replace("fifteen", '15',regex=True)\
.str.replace("(Twelve|twelve)", '12',regex=True)\
.str.replace("(,9|nine)", '9',regex=True)\
.str.replace("(,3|three|Three)", '3',regex=True)\
.str.replace("(,8|/8|eight|Eight|œ8)", '8',regex=True)\
.str.replace("(,4|/4|four|Four)", '4',regex=True)\
.str.replace("(,2|/2|two|Two|œ2)", '2',regex=True)\
.str.replace("(,7|/7|seven)", '7',regex=True)\
.str.replace("(,6|six)", '6',regex=True)\
.str.replace(",1", '1',regex=True)\
.str.replace("[a-zA-Z+]", '',regex=True)\
.str.replace("(',', | ','|/|,0|Â)", '')\
.str.replace(" ", '').apply(ast.literal_eval)

data_filtered['yrs'] = data_filtered['yrs'].astype(str)\
         .str.replace(".", ' ')\
.str.replace("+", ' ')\
.str.replace("(\d-)", '',regex=True)\
.str.replace("(-)", '',regex=True)\
.str.replace("(,1\d|,10)", '10',regex=True)\
.str.replace(",5", '5',regex=True)\
.str.replace(",9", '9',regex=True)\
.str.replace(",3", '3',regex=True)\
.str.replace("(,8|/8)", '8',regex=True)\
.str.replace("(,4|/4)", '4',regex=True)\
.str.replace("(,2|/2)", '2',regex=True)\
.str.replace("(,7|/7)", '7',regex=True)\
.str.replace(",6", '6',regex=True)\
.str.replace(",1", '1',regex=True)\
.str.replace("(',', | ','|/|,0)", '')\
.str.replace(" ", '').apply(ast.literal_eval)

yrs_exp = data_filtered['yrs']

data_filtered['yrs_exp'] = yrs_exp

data_filtered['yrs_exp2'] = np.where(data_filtered['yrs_exp'].astype(str)\
         .str.contains("\d"), data_filtered['yrs_exp'], '0')
data_filtered['yrs_exp2']

seniority_data = data_filtered[(data_filtered['yrs_exp2'] != '0')]
avgtest = seniority_data['yrs_exp2']
avgtest2 = [' '.join(i).split() for i in avgtest]
converted = [[int(num) for num in sub] for sub in avgtest2]
means = [np.mean([el for el in sublist if el > 0] or 0) for sublist in converted]
seniority_data['means'] = means
seniority_data['means'] = seniority_data['means'].astype('int64')

def not_applicable_seniority(df):
        
    if (df['means'] <= 3):
        return 'Jr'
    elif (df['means'] > 3 and df['means'] <= 10):
        return 'Mid'
    elif (df['means'] > 10 and df['means'] <= 25):
        return 'Sr'
    else:
        return 'Not Applicable'
    
seniority_data['means_class'] = seniority_data.apply(not_applicable_seniority, axis = 1)

def not_applicable_seniority_reclass(df):
        
    if (df['seniority'] == 'Not Applicable'):
        return df['means_class']
    else:
        return df['seniority']
    
seniority_data['seniority_v2'] = seniority_data.apply(not_applicable_seniority_reclass, axis = 1)

merged_table = data_filtered.merge(seniority_data[['seniority_v2']], how='left', left_index=True, right_index=True)
merged_table['seniority_v2']= merged_table[['seniority_v2']].fillna(value=0)
#merged_table

def not_applicable_seniority_merge(df):
        
    if (df['seniority_v2'] == 0):
        return df['seniority']
    else:
        return df['seniority_v2']

merged_table['seniority_v3'] = merged_table.apply(not_applicable_seniority_merge, axis = 1)
data_filtered['seniority_v3'] = merged_table['seniority_v3']
